In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, TensorDataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import time
import datetime
import random
from tqdm.auto import tqdm
# !pip install pyqlib
from Layers import FeatureExtractor, FactorEncoder, AlphaLayer, BetaLayer, FactorDecoder, FactorVAE, FatorPrior, FeatureMask
from dataset import StockDataset
from tqdm.auto import tqdm
from dataclasses import dataclass
from utils import test_args, load_model, generate_prediction_scores, single_pred
from  scipy.stats import spearmanr

In [10]:

args = test_args(run_name='None',
                num_factor=10, 
                normalize=True, 
                select_feature=False,
                hidden_size=20,
                num_latent=6,
                use_qlib=False)
factorVAE = load_model(args)
featureExtractor = FeatureExtractor(num_latent = args.num_latent, hidden_size =args.hidden_size)
featureMask = FeatureMask(num_latent=args.num_latent, hidden_size=args.num_latent)
# featureExtractor_model = "./best_models/factorExtractor_None_8_norm_True_char_False.pt"
featureMask_model = 'best_models/10/best_feat_mask_RationalStock_yearmonthenv_10.pt'
factorVAE_model = "best_models/10/best_factorVAE_RationalStock_yearmonthenv_10.pt"
factorVAE.load_state_dict(torch.load(factorVAE_model))
featureMask.load_state_dict(torch.load(featureMask_model))
# featureExtractor.load_state_dict(torch.load(featureExtractor_model))

<All keys matched successfully>

In [11]:
# index_columns = ['sh000001open', 'sh000001high', 'sh000001low', 'sh000001close',
#        'sh000001volume', 'sh000001next_open', 'sh000698open', 'sh000698high',
#        'sh000698low', 'sh000698close', 'sh000698volume', 'sh000698next_open',
#        'sz399001open', 'sz399001high', 'sz399001low', 'sz399001close',
#        'sz399001volume', 'sz399001next_open', 'sz399006open', 'sz399006high',
#        'sz399006low', 'sz399006close', 'sz399006volume', 'sz399006next_open']+[]

In [12]:
month = ["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec",]
year = [str(i) for i in range(1990,2025)]

In [13]:
index_columns =  month+ year

In [14]:
# index_columns = year+month

In [15]:
test_index = np.load("./data/test_index.npy")

NameError: name 'test_df' is not defined

In [ ]:
args.seq_length

In [ ]:
# test_df

In [ ]:
# test_df.drop(columns=index_columns,inplace=True)

In [ ]:
test_df = pd.read_pickle(f"./data/test_all.pkl")
# test_df = test_df[test_df.columns[:39]]
test_df.drop(columns=index_columns,inplace=True)
test_ds = StockDataset(test_df, test_index,args.batch_size, args.seq_length)
test_dataloader = DataLoader(test_ds, batch_size=4000, shuffle=False) 

In [ ]:
# test_df.set_index(["datetime","instrument"],inplace=True)
# test_df.sort_index(inplace=True)

In [ ]:
output = generate_prediction_scores(featureMask,factorVAE, test_dataloader, test_df, args)
#output.to_csv(f"./scores/{args.run_name}_{args.num_factor}_{args.normalize}_{args.select_feature}_{args.num_latent}_{args.hidden_size}.csv")

In [ ]:
# masks.index = 
output.index = test_df.index[test_index[:,-1]]

In [ ]:
output.index

In [ ]:
output


In [ ]:
# codes = output.index.codes
# new_levels = [pd.to_datetime(output.index.levels[0].astype(str),),output.index.levels[1]]
# new_index = pd.MultiIndex(levels=new_levels, codes=codes, names=output.index.names)
# output.index = new_index

In [ ]:
df_all = pd.read_pickle("./data/simpleusdataset.pkl")
# df_all.datetime = pd.to_datetime(df_all.datetime.astype(str))
# df_all.set_index(["datetime","instrument"],inplace=True)
# df_all.sort_index(inplace=True)

In [ ]:
test_df

In [ ]:
test_df

In [ ]:

a = pd.merge(test_df[["LABEL0"]],output,left_index=True,right_index=True)
# a = pd.merge(df_all,a,left_index=True,right_index=True)
# label_norm = pd.read_csv("../data/label_norm.csv")
# label_norm.datetime = pd.to_datetime(label_norm.datetime.astype(str))
# label_norm.set_index(["datetime","instrument"],inplace=True)
# label_norm.sort_index(inplace=True)
# total = pd.merge(a, label_norm, left_index=True, right_index=True)
# total.rename(columns={"LABEL0_x":"LABEL0"},inplace=True)
# total.rename(columns={"LABEL0_y":"next_close"},inplace=True)
# total.rename(columns={"close_x":"close"},inplace=True)
# total["tradable"] = True
# total["tradable"] = ~ ((total["open"] == total["high"]) &  (total["open"] == total["low"]) & (total["open"] == total["close"]))

In [ ]:
# total["tradable"] = total["tradable"].groupby("instrument").shift(-1)

In [ ]:
# total.dropna(inplace=True)

In [ ]:
a[["LABEL0","score"]].to_pickle("rs_us.pkl")

In [ ]:
total = a

In [ ]:
total["tradable"] = True

In [ ]:
total.rename(columns={"LABEL0":"label","score":"pred"},inplace=True)

In [ ]:
def calculate_rank_ic(df):
    # Rank transformation
    ranked_preds = df.score.rank(axis=0)
    ranked_actuals = df.LABEL0.rank(axis=0)

    # Standardize
    standardized_preds = (ranked_preds - ranked_preds.mean()) / ranked_preds.std()
    standardized_actuals = (ranked_actuals - ranked_actuals.mean()) / ranked_actuals.std()

    # Calculate individual Rank ICs
    individual_rank_ics = standardized_preds * standardized_actuals

    # Average over time periods
    rank = pd.concat([ranked_preds,ranked_actuals,individual_rank_ics],axis=1).sort_values("score")
    return individual_rank_ics,rank

In [ ]:
def back_test(all,number):
    asset = 1
    history = [1]
    new_potofolio_short = potofolio_short = new_potofolio = potofolio =  new_optimal_potofolio = optimal_potofolio = []
    df_trade_list = []
    profit_list = []
    # all["tradable"] = True

    for date, df in all.groupby("datetime"):
        trade_df =  df.loc[df.tradable].sort_values(by="pred",ascending=False).head(number)
        new_potofolio = trade_df.index.get_level_values("instrument")
        if len(potofolio)==0:
            fee = 0
        else:
            fee = len(set(new_potofolio) - set(potofolio))/len(potofolio) * 0.0015
        profit = trade_df["label"].mean()
        # profit_list.append()
        # fee = 0   

        long_asset =0.5 *asset* (1+profit-fee)
        potofolio = new_potofolio
        

        trade_df_short =  df.loc[df.tradable].sort_values(by="pred",ascending=True).head(number)
        new_potofolio_short = trade_df_short.index.get_level_values("instrument")
        if len(potofolio_short)==0:
            fee_short = 0
        else:
            fee_short = len(set(new_potofolio_short) - set(potofolio_short))/len(potofolio_short) * 0.0015
        profit_short = -trade_df_short["label"].mean()
        profit_list.append((profit-fee+profit_short-fee_short)/2)
        # fee = 0   
        
        short_asset =0.5*asset*(1+profit_short-fee_short)
        potofolio_short = new_potofolio_short
        asset = long_asset + short_asset
        df_trade_list.append(pd.concat([trade_df,trade_df_short]))
        history.append(asset)

    df_history = pd.DataFrame(history[1:],index=all.index.get_level_values("datetime").unique())
    df_history.rename(columns={0:"Cumulative_Returns"},inplace=True)
    df_history['Drawdown'] = df_history['Cumulative_Returns'].div(df_history['Cumulative_Returns'].cummax()) - 1
    df_history["return_rate"] = profit_list
    df_history['Drawdown'].min()

    return df_history

In [ ]:
hist = back_test(total,30)
hist

In [ ]:
plt.plot(hist["Cumulative_Returns"])
plt.xticks(rotation=45)

In [ ]:
def rankic(df):
    ic = df.groupby('datetime').apply(lambda df: df["label"].corr(df["pred"]))
    ric = df.groupby('datetime').apply(lambda df: df["label"].corr(df["pred"], method="spearman"))
    
    print({
            "IC": ic.mean(),
            "ICIR": ic.mean() / ic.std(),
            "Rank IC": ric.mean(),
            "Rank ICIR": ric.mean() / ric.std(),
        })
    return ric

In [ ]:
rankic(total)

In [ ]:
df_trade["LABEL0"].groupby("datetime").mean().sort_values(ascending=False).index[:20]

In [ ]:
rankics = total.groupby("datetime")["rankic"].mean()
rankics.mean()

In [ ]:
ranicir = rankics.mean()/rankics.std()
ranicir

In [ ]:
ic = total.groupby('datetime').apply(lambda df: df["LABEL0"].corr(df["score"]))
ric = total.groupby('datetime').apply(lambda df: df["LABEL0"].corr(df["score"], method="spearman"))

In [ ]:
{
            "IC": ic.mean(),
            "ICIR": ic.mean() / ic.std(),
            "Rank IC": ric.mean(),
            "Rank ICIR": ric.mean() / ric.std(),
        }

In [ ]:
history

In [ ]:
# df.sort_values(by="change_est",ascending=False).head(10)

In [ ]:
set(potofolio) - set(new_potofolio)

In [ ]:
import akshare as ak
stock_zh_index_daily_df = ak.stock_zh_index_daily(symbol="sh000001")
stock_zh_index_daily_df.date = pd.to_datetime(stock_zh_index_daily_df.date)
stock_zh_index_daily_df.set_index("date",inplace=True)
print(stock_zh_index_daily_df)

In [ ]:
from pprint import pprint
from qlib.utils.time import Freq
from qlib.contrib.evaluate import risk_analysis
FREQ = "day"
analysis_freq = "{0}{1}".format(*Freq.parse(FREQ))

In [ ]:

# analysis = dict()
# analysis["excess_return_without_cost"] = risk_analysis(
#     analysis_report["return"] - analysis_report["bench"], freq=analysis_freq
# )
# analysis["excess_return_with_cost"] = risk_analysis(
#     analysis_report["return"] - analysis_report["bench"] - analysis_report["cost"], freq=analysis_freq
# )

# analysis_df = pd.concat(analysis)
# pprint(analysis_df)

In [ ]:
potofolio
## Get Bench
import akshare as ak
stock_zh_index_daily_df = ak.stock_zh_index_daily(symbol="sh000001")
stock_zh_index_daily_df.date = pd.to_datetime(stock_zh_index_daily_df.date)
stock_zh_index_daily_df.set_index("date",inplace=True)
print(stock_zh_index_daily_df)

In [ ]:
bench = ((stock_zh_index_daily_df.close.shift(1)-stock_zh_index_daily_df.close)/stock_zh_index_daily_df.close)
bench = stock_zh_index_daily_df[["close"]]
analysis_report = pd.merge(analysis_report,bench,left_index=True,right_index=True)

analysis_report.rename(columns={"close":"sz_rate"},inplace=True)


analysis_report["sz_return"] = analysis_report.sz_rate/analysis_report.sz_rate.iloc[0]
df_trade.drop(columns=["LABEL0","score",],inplace=True)
analysis_report

In [ ]:
analysis_report.index

In [ ]:
analysis_report.index.names = ["datetime"]

In [ ]:
## MaxDrawdown
df_history = pd.DataFrame(history[1:],index=total.index.get_level_values("datetime").unique())
df_history.rename(columns={0:"Cumulative_Returns"},inplace=True)
df_history['Drawdown'] = df_history['Cumulative_Returns'].div(df_history['Cumulative_Returns'].cummax()) - 1
df_history['Drawdown'].min()

plt.plot(df_history.index, df_history['Drawdown'], label='Drawdown', color='red')
plt.fill_between(df_history.index, df_history['Drawdown'], alpha=0.3, color='red')
plt.title('Drawdown Over Time')
plt.xlabel('Date')
plt.ylabel('Drawdown')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
## Annualized return
year = len(history)/252
annualized_return = history[-1]**(1/year)-1
annualized_return

In [ ]:
## Sharp Ratio
import akshare as ak
bond_zh_us_rate_df = ak.bond_zh_us_rate(start_date="19901219")

risk_free_rate = bond_zh_us_rate_df.loc[(pd.to_datetime(bond_zh_us_rate_df["日期"])>pd.to_datetime("20200101"))&(pd.to_datetime(bond_zh_us_rate_df["日期"])<("20221101")),"中国国债收益率2年"].mean()/100
standard_deviation_of_investment = analysis_report["return_rate"].std()* np.sqrt(252)
standard_deviation_of_investment
sharpe_ratio = (annualized_return - risk_free_rate) / standard_deviation_of_investment
sharpe_ratio

In [ ]:
analysis_report

In [ ]:
analysis_report

In [ ]:
# ## Imformation Ratio
# ir = (analysis_report["return_rate"] - analysis_report["sz_return"]).mean()/analysis_report["return_rate"].std()
# ir

In [ ]:
len(df_history.index)

In [ ]:


import matplotlib.pyplot as plt
plt.plot(df_history.index,history[1:],label="Ours")
plt.plot(df_history.index,bench_hist[1:],label="Bench")
plt.plot(df_history.index,analysis_report.sz_return.values,label="Shanghai SE Composite Index")
plt.legend()

In [ ]:
df.loc[df.index.get_level_values("instrument").isin(potofolio)]
# for date, df in ((total["prediction"] - total["close"])/total["close"]).groupby("datetime"):
#     print(date)
#     print(df.sort_values(ascending=False).head(10))


# codes = output.index.codes
# new_levels = [pd.to_datetime(output.index.levels[0].astype(str),),output.index.levels[1]]
# new_index = pd.MultiIndex(levels=new_levels, codes=codes, names=output.index.names)
# output.index = new_index


# p = single_pred(factorVAE, test_dataloader, test_ds.df, args)
from pprint import pprint
# import qlib.contrib.report as qcr
# import qlib
# import pandas as pd
from qlib.utils.time import Freq
# from qlib.utils import flatten_dict
# from qlib.backtest import backtest, executor
# from qlib.contrib.evaluate import backtest_daily
from qlib.contrib.evaluate import risk_analysis
# from qlib.contrib.strategy import TopkDropoutStrategy

# qlib.init()

# CSI300_BENCH = "SH000300"
FREQ = "day"
# STRATEGY_CONFIG = {
#     "topk": 50,
#     "n_drop": 5,
#     # pred_score, pd.Series
#     "signal": output,
# }

# EXECUTOR_CONFIG = {
#     "time_per_step": "day",
#     "generate_portfolio_metrics": True,
# }

# backtest_config = {
#     "start_time": "2020-02-06",
#     "end_time": "2022-11-01",
#     "account": 100000000,
#     # "benchmark": CSI300_BENCH,
#     "exchange_kwargs": {
#         "freq": FREQ,
#         "limit_threshold": 0.095,
#         "deal_price": "close",
#         "open_cost": 0.0005,
#         "close_cost": 0.0015,
#         "min_cost": 5,
#     },
# }
# # strategy object
# strategy_obj = TopkDropoutStrategy(**STRATEGY_CONFIG)
# # executor object
# executor_obj = executor.SimulatorExecutor(**EXECUTOR_CONFIG)

# # backtest
# portfolio_metric_dict, indicator_dict = backtest(executor=executor_obj, strategy=strategy_obj, **backtest_config)
# analysis_freq = "{0}{1}".format(*Freq.parse(FREQ))

# # backtest info
# report_normal_df, positions_normal = portfolio_metric_dict.get(analysis_freq)
analysis_freq = "{0}{1}".format(*Freq.parse(FREQ))
analysis_report.index

analysis_report["turnover"] = 1
analysis_report

In [ ]:
# analysis_report.drop(["bench_rate","sz_rate","sz_return"],axis=1,inplace =True)

In [ ]:
analysis_report.index.strftime("%Y-%m-%d")
import qlib.contrib.report as qcr
from qlib.contrib.report import analysis_position, analysis_model
print(qcr.GRAPH_NAME_LIST)
analysis_position.report_graph(analysis_report)

In [ ]:

analysis = dict()
analysis["excess_return_without_cost"] = risk_analysis(
    analysis_report["return"] - analysis_report["bench"], freq=analysis_freq
)
analysis["excess_return_with_cost"] = risk_analysis(
    analysis_report["return"] - analysis_report["bench"] - analysis_report["cost"], freq=analysis_freq
)

analysis_df = pd.concat(analysis)  # type: pd.DataFrame
analysis_position.risk_analysis_graph(analysis_df, analysis_report)

In [ ]:
analysis = dict()
analysis["excess_return_without_cost"] = risk_analysis(
    analysis_report["return"] - analysis_report["bench"], freq=analysis_freq
)
analysis["excess_return_with_cost"] = risk_analysis(
    analysis_report["return"] - analysis_report["bench"] - analysis_report["cost"], freq=analysis_freq
)

analysis_df = pd.concat(analysis)
pprint(analysis_df)
potofolio